# Dirichlet clustered mixture models for force unfolding peaks

In [ ]:
include("gibbs.jl");
plt = palette(:default);

In [ ]:
rng = MersenneTwister();

## Load the data

We enumerate the $\mathrm{Ca}^{2+}$ states in the order p000 p001 p011 p111 p010 p100 p101 p110.

This is slightly the reverse of what was done in the `Ca2+ Site Probabilities.ipynb` notebook where there it was p010 p100 p101 p110 p000 p001 p011 p111. We swap the last four and first four in final csv export for use in other notebooks.

In [ ]:
df = CSV.read("Force peaks grouped by unfolding.csv",DataFrame);
replace!(df[!,"Unfolding"],"Ca2+ coordination"=>"Ca²⁺ coordination");
sort!(df,"Construct",rev=true)

#### Augment by $\mathrm{Ca}^{2+}$ state id
We add a column called Caid that gives integer id for p000 p001 p011 p111 p010 p100 p101 p110.

In [ ]:
function Caid(v::Vector{S}) where S<:AbstractString
    # p000 p001 p011 p111 p010 p100 p101 p110
    if v == ["-","-","-"]
        return 1
    elseif v == ["-","-","Ca2+"]
        return 2
    elseif v == ["-","Ca2+","Ca2+"]
        return 3
    elseif v == ["Ca2+","Ca2+","Ca2+"]
        return 4
    elseif v == ["-","Ca2+","-"]
        return 5
    elseif v == ["Ca2+","-","-"]
        return 6
    elseif v == ["Ca2+","-","Ca2+"]
        return 7
    elseif v == ["Ca2+","Ca2+","-"]
        return 8
    end
end;

In [ ]:
transform!(df,["Site 1","Site 2","Site 3"]=> ( (x,y,z)->[ Caid([x[i],y[i],z[i]]) for i=1:length(x)] ) =>"Caid")
df = hcat(df[!,1:4],df[!,end],df[!,5:end-1]); 
rename!(df,"x1"=>"Caid");

#### Group by construct type
The $x$ indicates the EC19-20 values since they are from the originating mixture distribution and are ancestral to the $y$, indicating EC17-25 values, which are sampled from the minimum order statistic for the $x$'s.

In [ ]:
gdf = groupby(df,"Construct");
df_x = DataFrame(gdf[("EC19-20",)]);
df_y = DataFrame(gdf[("EC17-25",)]);

In [ ]:
μ = sum(df_x[!,end])/nrow(df_x); σsq = sum( (df_x[!,end].-μ).^2 )/nrow(df_x)
println("EC19-20 sample force (μ,σ²)=($(μ),$(σsq))")

In [ ]:
p1 = histogram(df_x[!,end],normalize=:pdf,
          xlabel="fpeak (pN)",ylabel="density",labels="",title="EC19-20",
          linewidth=0,linecolor=:white,size=(450,300))
scatter!(df_x[!,end],fill(0.,nrow(df_x)),marker=:o,labels="",markersize=6)

## Intuition behind our probability model for clustering
- We say our force data is comprised of an unknown number of unfolding events and each unfolding event has a normally distributed force value.
- Each $\mathrm{Ca}^{2+}$ state has its own probability vector for how likely each unfolding event is to occur.
- Some of the $\mathrm{Ca}^{2+}$ states may share the same probability vector (sparsity principle).
- The EC17-25 force is the minimum order statistic for its states associated unfolding force distribution.

So we're trying to cluster the $\mathrm{Ca}^{2+}$ states unfolding forces into mixtures of normal distributions. The number of component normals (corresponds to # of unfolding events) and the number of distinct mixture profiles (corresponds to # of force different $\mathrm{Ca}^{2+}$ states) are estimated. 

## Defining the probabilistic model for clustering
The probability model is
1. Unfolding event $i$ force mean <br>
$\mu_i \sim_{\mathrm{iid}} \mathcal{N}\left(900,200\right)$.
2. Unfolding event $i$ force variance <br>
$\log_{10}\sigma^2_i \sim \mathrm{Unif}\left[0,\log_{10}90000\right]$.
3. Concentration hyperparameter for the number of unfolding events in the data<br>
$\alpha\sim\Gamma\left(1,1\right)1_{\left[0.1,\infty\right)}$.
4. Candidate mixture weights/unfolding event profiles for the relative frequency of any $\mathrm{Ca}^{2+}$ state's unfolding events (capped at 8 events and up to 8 candidate profiles/where each state has its own) <br>
$w_i \left.\right| \alpha \sim_{\mathrm{iid}} \mathrm{Dirichlet}\big(\alpha\left(\frac{1}{8},\ldots,\frac{1}{8}\right)\big)$.
5. Concentration parameter for how many unfolding event profiles are actually present across $\mathrm{Ca}^{2+}$ states <br>
$\beta\sim\Gamma\left(1,1\right)1_{\left[0.1,\infty\right)}$.
6. Pmf for how the unfolding event profiles are distributed throughout the $\mathrm{Ca}^{2+}$ states <br>
$\theta\left.\right|\beta \sim\mathrm{Dirichlet}\left(\beta\left(\frac{1}{8},\ldots,\frac{1}{8}\right)\right)$.
7. Assignment of $\mathrm{Ca}^{2+}$ state $i$ to a profile <br>
$j_i\left.\right|\theta \sim_{\mathrm{iid}}\mathrm{Categorical}\left(\theta\right)$.
8. Observed unfolding force at $\mathrm{Ca}^{2+}$ state $i$ <br>
$x_i\left.\right|j_i,w_{j_i},\vec{\mu},\vec{\sigma} \sim \sum_k w_{kj_i}\mathrm{pdf}_{\mathcal{N}\left(\mu_k,\sigma_k\right)}(x_i)$.
9. Observed EC17-25 unfolding force at $\mathrm{Ca}^{2+}$ state $i$ with $n$ the number of linker regions<br>
$y_i \left.\right|j_i,w_{j_i},\vec{\mu},\vec{\sigma} \sim n \left(1-\mathrm{cdf}_{\sum_k w_{kj_i}\mathrm{pdf}_{\mathcal{N}\left(\mu_k,\sigma_k\right)}}(y_i)\right)^{n-1}\mathrm{pdf}_{\sum_k w_{kj_i}\mathrm{pdf}_{\mathcal{N}\left(\mu_k,\sigma_k\right)}}(y_i)$.

The last density is that for a minimum order statistic in $n$ trials whose base random variable is described by the pdf. The scales in 1. and 2. were suggested by the scale of the sample data. 

As usual, we estimate our unknown quantities by conditioning on the observed ones and forming the posterior. We are primarily interested in the posterior distributions of the unfolding force distributions for each $\mathrm{Ca}^{2+}$ state.

## MCMC sample the posterior
#### Extract values of conditioning variables
We first grab the data variables that we are conditioning on, ordered like the Caid function: p000 p001 p011 p111 p010 p100 p101 p110.

In [ ]:
tmp = sort(df_x,"Caid"); xs = tmp[:,end];
tmp = sort(df_y,"Caid"); ys = tmp[:,end];

### Define routine to initialize MCMC samples
Parameters are ordered like $p = \left(\vec{\mu},\log_{10}\vec{\sigma}^2,\alpha,\vec{w}_1,\ldots,\vec{w}_8,\beta,\vec{\theta},\vec{j}\right)$.

In [ ]:
# initialize by proposing from prior
function gibbssmp_p0!(rng,p0)
    # sample means
    for ℓ=1:8
        μ = 900+200*randn(rng)
        set_μ!(p0,μ;k=ℓ)
    end
    
    # sample variances
    for ℓ=1:8
        log10σsq = log10(90000)*rand(rng)
        set_log10σsq!(p0,log10σsq;k=ℓ)
    end
    
    # sample α
    Γ = Gamma(1,1)
    α = rand(rng,Γ); flagfd = false
    while !flagfd
        α = rand(rng,Γ)
        flagfd = α >= 0.1 ? true : false
    end
    set_α!(p0,α)
    
    # sample ws
    P0 = get_P0()
    Dα = Dirichlet(α*P0)
    for ℓ=1:8
        wi = rand(rng,Dα)
        set_w!(p0,wi;k=ℓ)
    end
    
    # sample β
    β = rand(rng,Γ); flagfd = false
    while !flagfd
        β = rand(rng,Γ)
        flagfd = β >= 0.1 ? true : false
    end
        
    set_β!(p0,β)
    
    # set θ
    Dβ = Dirichlet(β*P0)
    θ = rand(rng,Dβ)
    set_θ!(p0,θ)
    
    # set j
    C = Categorical(θ)
    for ℓ=1:8
        ji = rand(rng,C)
        set_j!(p0,ji;k=ℓ)
    end
end;

### Define Metropolis within Gibbs kernels for the conditionals

#### Ancillary routines to deal with small roundoff errors

In [ ]:
function mylog(x)
    if x>=0
        return log(x)
    elseif abs(x)>1e-9
        @warn "Taking log of $(x)."
    end
    
    return -Inf
end   

function mypos(x)
    if x>=0
        return x
    elseif abs(x)>1e-9
        @warn "$(x) should be nonnegative."
    end
    
    val = 0*x
    return val
end;

#### Auxilliary routines for parameter management
Parameters are ordered like $p = \left(\vec{\mu},\log_{10}\vec{\sigma}^2,\alpha,\vec{w}_1,\ldots,\vec{w}_8,\beta,\vec{\theta},\vec{j}\right)$.

In [ ]:
get_P0() = SVector{8,Float64}([1/8,1/8,1/8,1/8,1/8,1/8,1/8,1/8]);

We write the routines for vectors or components based on how much the Gibbs sampler updates at once per its cycle kernels on conditionals.

In [ ]:
# means for the unfolding events
get_μ(p0;k) = p0[k]
function set_μ!(p0,μ;k)
    p0[k] = μ
end

# variances for the unfolding events
get_log10σsq(p0;k) = p0[8+k]
function set_log10σsq!(p0,log10σsq;k)
    p0[8+k] = log10σsq
end

# concentration parameter for # of events in candidate unfolding profiles
get_α(p0) = p0[16+1]
function set_α!(p0,α)
    p0[16+1] = α
end

# candidate unfolding profiles
get_w(p0;k) = SVector{8,Float64}(p0[17+8*(k-1)+1:17+8*k])
function set_w!(p0,w;k)
    p0[17+8*(k-1)+1:17+8*k] .= w
end

# concentration parameter for # of unfolding profiles across Ca2+ states
get_β(p0) = p0[17+64+1]
function set_β!(p0,β)
    p0[17+64+1] = β
end

# pmf for unfolding profiles distributed across Ca2+ states
get_θ(p0) = SVector{8,Float64}(p0[17+64+1+1:17+64+1+8])
function set_θ!(p0,θ)
    p0[17+64+1+1:17+64+1+8] .= θ
end

# assignment of Ca2+ state to an unfolding profile
get_j(p0;k) = p0[17+64+1+8+k]
function set_j!(p0,j;k)
    p0[17+64+1+8+k] = j
end;

#### Gibbs sampler for conditional of means of unfolding event forces
The relevant factors are
1. Unfolding event $i$ force mean <br>
$\mu_i \sim_{\mathrm{iid}} \mathcal{N}\left(900,200\right)$.
2. Observed unfolding force at $\mathrm{Ca}^{2+}$ state $i$ <br>
$x_i\left.\right|j_i,w_{j_i},\vec{\mu},\vec{\sigma} \sim \sum_k w_{kj_i}\mathrm{pdf}_{\mathcal{N}\left(\mu_k,\sigma_k\right)}(x_i)$.
3. Observed EC17-25 unfolding force at $\mathrm{Ca}^{2+}$ state $i$ with $n$ the number of linker regions<br>
$y_i \left.\right|j_i,w_{j_i},\vec{\mu},\vec{\sigma} \sim n \left(1-\mathrm{cdf}_{\sum_k w_{kj_i}\mathrm{pdf}_{\mathcal{N}\left(\mu_k,\sigma_k\right)}}(y_i)\right)^{n-1}\mathrm{pdf}_{\sum_k w_{kj_i}\mathrm{pdf}_{\mathcal{N}\left(\mu_k,\sigma_k\right)}}(y_i)$.

In [ ]:
# compute unnormalized conditional density for μᵢ
function logπ_μ(p0;k,xs=xs,ys=ys)
    # 1.
    μi = get_μ(p0;k=k)
    val = -0.5*(μi-900)^2/40000
    
    # 2.
    #  Get assignment of Ca2+ states to profiles
    js = @SVector [Int64(get_j(p0;k=i)) for i=1:8]
    
    #  Get candidate event frequency profiles
    ws = [get_w(p0;k=i) for i=1:8]

    #  Get means
    μs = @SVector [get_μ(p0;k=i) for i=1:8]
    
    #  Get variances 
    σs = @SVector [get_log10σsq(p0;k=i) for i=1:8]
    σs = .√(10 .^(σs))
    
    #  Define normals
    Ns = [Normal(μs[i],σs[i]) for i=1:8]
    
    #  Aggregate contribution of mixture pdfs over the observed unfoldings for EC19-20
    tmp = 0.
    for ℓ=1:8
        tmp = tmp + log( sum( ws[js[ℓ]].*pdf.(Ns,xs[ℓ]) ) )
    end
    
    val = val + tmp
    
    # 3.
    #  Aggregate contribution of mixtures over observed unfolding for EC17-25
    tmp = 0.
    for ℓ=1:4
        tmp = tmp + (8-1)*mylog( 1 - sum( ws[js[ℓ]].*cdf.(Ns,ys[ℓ]) ) ) + log( sum( ws[js[ℓ]].*pdf.(Ns,ys[ℓ]) ) )
    end
    
    val = val + tmp
    
    return val
end;

In [ ]:
function gibbssmp_μ!(rng,p0)
    # cycle MH random walk on each of the event means
    aptrate = 0
    for ℓ=1:8
        # get current value
        μ0 = get_μ(p0;k=ℓ); 
        
        # random walk propose new mean
        μ = μ0 + 5*randn(rng)
        
        # compute mh ratio
        logπ0 = logπ_μ(p0;k=ℓ)
        set_μ!(p0,μ;k=ℓ); logπ = logπ_μ(p0;k=ℓ)
        
        coin = rand(rng) |> log
        if coin <= logπ-logπ0
            # accept
            aptrate += 1
        else
            # reject
            set_μ!(p0,μ0;k=ℓ)
        end
    end
    
    return aptrate/8
end;

#### Gibbs sampler for conditional of standard deviations of unfolding event forces
The relevant factors are
1. Unfolding event $i$ force variance <br>
$\log_{10}\sigma^2_i \sim \mathrm{Unif}\left[0,\log_{10}90000\right]$.
2. Observed unfolding force at $\mathrm{Ca}^{2+}$ state $i$ <br>
$x_i\left.\right|j_i,w_{j_i},\vec{\mu},\vec{\sigma} \sim \sum_k w_{kj_i}\mathrm{pdf}_{\mathcal{N}\left(\mu_k,\sigma_k\right)}(x_i)$.
3. Observed EC17-25 unfolding force at $\mathrm{Ca}^{2+}$ state $i$ with $n$ the number of linker regions<br>
$y_i \left.\right|j_i,w_{j_i},\vec{\mu},\vec{\sigma} \sim n \left(1-\mathrm{cdf}_{\sum_k w_{kj_i}\mathrm{pdf}_{\mathcal{N}\left(\mu_k,\sigma_k\right)}}(y_i)\right)^{n-1}\mathrm{pdf}_{\sum_k w_{kj_i}\mathrm{pdf}_{\mathcal{N}\left(\mu_k,\sigma_k\right)}}(y_i)$.

In [ ]:
# compute unnormalized conditional density for log₁₀σᵢ²
function logπ_log10σsq(p0;k,xs=xs,ys=ys)
    # 1.
    log10σsq = get_log10σsq(p0;k=k)
    if (log10σsq < 0)||(log10σsq > log10(90000))
        return -Inf
    end
    val = 0.
    
    # 2.
    #  Get assignment of Ca2+ states to profiles
    js = @SVector [Int64(get_j(p0;k=i)) for i=1:8]
    
    #  Get candidate event frequency profiles
    ws = [get_w(p0;k=i) for i=1:8]

    #  Get means
    μs = @SVector [get_μ(p0;k=i) for i=1:8]
    
    #  Get variances 
    σs = @SVector [get_log10σsq(p0;k=i) for i=1:8]
    σs = .√(10 .^(σs))
    
    #  Define normals
    Ns = [Normal(μs[i],σs[i]) for i=1:8]
    
    #  Aggregate contribution of mixture pdfs over the observed unfoldings for EC19-20
    tmp = 0.
    for ℓ=1:8
        tmp = tmp + log( sum( ws[js[ℓ]].*pdf.(Ns,xs[ℓ]) ) )
    end
    
    val = val + tmp
    
    # 3.
    #  Aggregate contribution of mixtures over observed unfolding for EC17-25
    tmp = 0.
    for ℓ=1:4
        tmp = tmp + (8-1)*mylog( 1 - sum( ws[js[ℓ]].*cdf.(Ns,ys[ℓ]) ) ) + log( sum( ws[js[ℓ]].*pdf.(Ns,ys[ℓ]) ) )
    end
    
    val = val + tmp
    
    return val
end;

In [ ]:
function gibbssmp_log10σsq!(rng,p0)
    # cycle MH random walk on each of the event standard deviations
    aptrate = 0
    for ℓ=1:8
        # get current value
        log10σsq0 = get_log10σsq(p0;k=ℓ); 
        
        # random walk propose new mean
        log10σsq = log10σsq0 + 0.5*randn(rng)
        
        # compute mh ratio
        logπ0 = logπ_log10σsq(p0;k=ℓ)
        set_log10σsq!(p0,log10σsq;k=ℓ); logπ = logπ_log10σsq(p0;k=ℓ)
        
        coin = rand(rng) |> log
        if coin <= logπ-logπ0
            # accept
            aptrate += 1
        else
            # reject
            set_log10σsq!(p0,log10σsq0;k=ℓ)
        end
    end
    
    return aptrate/8
end;

#### Gibbs sampler for conditional of the concentration parameter for number of unfolding events
The relevant factors are
1. Concentration hyperparameter for the number of unfolding events in the data<br>
$\alpha\sim\Gamma\left(1,1\right)1_{\left[0.1,\infty\right)}$.
2. Candidate mixture weights/unfolding event profiles for the relative frequency of any $\mathrm{Ca}^{2+}$ state's unfolding events (capped at 8 events and up to 8 candidate profiles/where each state has its own) <br>
$w_i \left.\right| \alpha \sim_{\mathrm{iid}} \mathrm{Dirichlet}\big(\alpha\left(\frac{1}{8},\ldots,\frac{1}{8}\right)\big)$.

In [ ]:
# compute unnormalized density for α
function logπ_α(p0)
    # 1.
    α = get_α(p0)
    
    Γ = Gamma(1,1)
    val = logpdf(Γ,α)
    if val == -Inf
        return val
    end
    
    if α < 0.1
        return -Inf
    end
    
    # 2. 
    #  Get candidate event frequency profiles
    ws = [get_w(p0;k=i) for i=1:8]
    
    P0 = get_P0()
    D = Dirichlet(α*P0)
    
    for ℓ=1:8
        val = val + logpdf(D,ws[ℓ])
    end
    
    return val
end;

In [ ]:
function gibbssmp_α!(rng,p0)
    aptrate = 0
    # get current value
    α0 = get_α(p0)
    
    # Propose by random walk
    α = α0 + 0.1*randn(rng)
    
    # compute mh ratio
    logπ0 = logπ_α(p0)
    set_α!(p0,α); logπ = logπ_α(p0)
    
    coin = rand(rng) |> log
    if coin <= logπ-logπ0
        # accept
        aptrate += 1
    else
        # reject
        set_α!(p0,α0)
    end
    
    return aptrate
end;

#### Gibbs sampler for conditional of mixture profiles of unfolding events
The relevant factors are
1. Candidate mixture weights/unfolding event profiles for the relative frequency of any $\mathrm{Ca}^{2+}$ state's unfolding events (capped at 8 events and up to 8 candidate profiles/where each state has its own) <br>
$w_i \left.\right| \alpha \sim_{\mathrm{iid}} \mathrm{Dirichlet}\big(\alpha\left(\frac{1}{8},\ldots,\frac{1}{8}\right)\big)$.
2. Observed unfolding force at $\mathrm{Ca}^{2+}$ state $i$ <br>
$x_i\left.\right|j_i,w_{j_i},\vec{\mu},\vec{\sigma} \sim \sum_k w_{kj_i}\mathrm{pdf}_{\mathcal{N}\left(\mu_k,\sigma_k\right)}(x_i)$.
3. Observed EC17-25 unfolding force at $\mathrm{Ca}^{2+}$ state $i$ with $n$ the number of linker regions<br>
$y_i \left.\right|j_i,w_{j_i},\vec{\mu},\vec{\sigma} \sim n \left(1-\mathrm{cdf}_{\sum_k w_{kj_i}\mathrm{pdf}_{\mathcal{N}\left(\mu_k,\sigma_k\right)}}(y_i)\right)^{n-1}\mathrm{pdf}_{\sum_k w_{kj_i}\mathrm{pdf}_{\mathcal{N}\left(\mu_k,\sigma_k\right)}}(y_i)$.

In [ ]:
function logπ_w(p0;k,xs=xs,ys=ys)
    # 1.
    α =  get_α(p0)
    wi = get_w(p0;k=k)
    
    P0 = get_P0()
    D = Dirichlet(α*P0)
    val = logpdf(D,wi)
    
    # 2.
    #  Get assignment of Ca2+ states to profiles
    js = @SVector [Int64(get_j(p0;k=i)) for i=1:8]
    
    #  Get candidate event frequency profiles
    ws = [get_w(p0;k=i) for i=1:8]

    #  Get means
    μs = @SVector [get_μ(p0;k=i) for i=1:8]
    
    #  Get variances 
    σs = @SVector [get_log10σsq(p0;k=i) for i=1:8]
    σs = .√(10 .^(σs))
    
    #  Define normals
    Ns = [Normal(μs[i],σs[i]) for i=1:8]
    
    #  Aggregate contribution of mixture pdfs over the observed unfoldings for EC19-20
    tmp = 0.
    for ℓ=1:8
        tmp = tmp + log( sum( ws[js[ℓ]].*pdf.(Ns,xs[ℓ]) ) )
    end
    
    val = val + tmp
    
    # 3.
    #  Aggregate contribution of mixtures over observed unfolding for EC17-25
    tmp = 0.
    for ℓ=1:4
        tmp = tmp + (8-1)*mylog( 1 - sum( ws[js[ℓ]].*cdf.(Ns,ys[ℓ]) ) ) + log( sum( ws[js[ℓ]].*pdf.(Ns,ys[ℓ]) ) )
    end
    
    val = val + tmp
    
    return val
end;

In [ ]:
function gibbssmp_w!(rng,p0)
    aptrate = 0
    # cycle MH proposing from conditonal prior
    α0 = get_α(p0)
    P0 = get_P0()
    D = Dirichlet(α0*P0)
    
    for ℓ=1:8
        wi0 = get_w(p0;k=ℓ)
        
        # conditional prior propose wi | α
        wi = rand(rng,D)
        
        # compute mh ratio
        logπ0 = logπ_w(p0;k=ℓ)
        set_w!(p0,wi;k=ℓ); logπ = logπ_w(p0;k=ℓ)
        
        coin = rand(rng) |> log
        if coin <= (logπ + logpdf(D,wi0))-(logπ0 + logpdf(D,wi))
            # accept
            aptrate += 1
        else
            # reject
            set_w!(p0,wi0;k=ℓ)
        end
    end
    
    return aptrate/8
end;

#### Gibbs sampler for concentration parameter of how many unfolding event profiles are present across $\mathrm{Ca}^{2+}$ states
The relevant factors are
1. Concentration parameter for how many unfolding event profiles are actually present across $\mathrm{Ca}^{2+}$ states <br>
$\beta\sim\Gamma\left(1,1\right)1_{\left[0.1,\infty\right)}$.
2. Pmf for how the unfolding event profiles are distributed throughout the $\mathrm{Ca}^{2+}$ states <br>
$\theta\left.\right|\beta \sim\mathrm{Dirichlet}\left(\beta\left(\frac{1}{8},\ldots,\frac{1}{8}\right)\right)$.

In [ ]:
function logπ_β(p0)
    # 1.
    β = get_β(p0)
    
    Γ = Gamma(1,1)
    val = logpdf(Γ,β)
    if val == -Inf
        return val
    end
    
    if β < 0.1
        return -Inf
    end
    
    # 2. 
    θ = get_θ(p0)
    
    P0 = get_P0()
    D = Dirichlet(β*P0)
    
    val = val + logpdf(D,θ)
    
    return val
end;

In [ ]:
function gibbssmp_β!(rng,p0)
    aptrate = 0
    # sample by MH random walk
    β0 = get_β(p0)
    
    # propose by random walk
    β = β0 + 0.1*randn(rng)
    
    # compute mh ratio
    logπ0 = logπ_β(p0)
    set_β!(p0,β); logπ = logπ_β(p0)
    
    coin = rand(rng) |> log
    if coin <= logπ - logπ0
        # accept
        aptrate += 1
    else
        # reject
        set_β!(p0,β0)
    end
    
    return aptrate
end;

#### Gibbs conditional sample the pmf for how unfolding profiles are distributed through $\mathrm{Ca}^{2+}$ states
The relevant factors are
1. Pmf for how the unfolding event profiles are distributed throughout the $\mathrm{Ca}^{2+}$ states <br>
$\theta\left.\right|\beta \sim\mathrm{Dirichlet}\left(\beta\left(\frac{1}{8},\ldots,\frac{1}{8}\right)\right)$.
2. Assignment of $\mathrm{Ca}^{2+}$ state $i$ to a profile <br>
$j_i\left.\right|\theta \sim_{\mathrm{iid}}\mathrm{Categorical}\left(\theta\right)$.

In [ ]:
# Gibbs sample by conjugate prior
function gibbssmp_θ!(rng,p0)
    β = get_β(p0)
    P0 = get_P0()
    
    # compute the frequency vector of how states were assigned to profiles
    #  Get assignment of Ca2+ states to profiles
    js = @SVector [Int64(get_j(p0;k=i)) for i=1:8]
    
    η = @SVector [sum(js.==i) for i=1:8]
    
    # sample using conjugate prior
    D = Dirichlet(β*P0+η)
    θ = rand(rng,D)
    
    set_θ!(p0,θ)
end;

#### Gibbs conditional sample the assignment of $\mathrm{Ca}^{2+}$ states to an unfolding profile
The relevant factors are
1. Assignment of $\mathrm{Ca}^{2+}$ state $i$ to a profile <br>
$j_i\left.\right|\theta \sim_{\mathrm{iid}}\mathrm{Categorical}\left(\theta\right)$.
2. Observed unfolding force at $\mathrm{Ca}^{2+}$ state $i$ <br>
$x_i\left.\right|j_i,w_{j_i},\vec{\mu},\vec{\sigma} \sim \sum_k w_{kj_i}\mathrm{pdf}_{\mathcal{N}\left(\mu_k,\sigma_k\right)}(x_i)$.
3. Observed EC17-25 unfolding force at $\mathrm{Ca}^{2+}$ state $i$ with $n$ the number of linker regions<br>
$y_i \left.\right|j_i,w_{j_i},\vec{\mu},\vec{\sigma} \sim n \left(1-\mathrm{cdf}_{\sum_k w_{kj_i}\mathrm{pdf}_{\mathcal{N}\left(\mu_k,\sigma_k\right)}}(y_i)\right)^{n-1}\mathrm{pdf}_{\sum_k w_{kj_i}\mathrm{pdf}_{\mathcal{N}\left(\mu_k,\sigma_k\right)}}(y_i)$.

In [ ]:
# Sample by its conditional via explicit normalization of pmf
function gibbssmp_j!(rng,p0;xs=xs,ys=ys)
    #  Get candidate event frequency profiles
    ws = [get_w(p0;k=i) for i=1:8]

    #  Get means
    μs = @SVector [get_μ(p0;k=i) for i=1:8]
    
    #  Get variances 
    σs = @SVector [get_log10σsq(p0;k=i) for i=1:8]
    σs = .√(10 .^(σs))
    
    #  Define normals
    Ns = [Normal(μs[i],σs[i]) for i=1:8]
    
    # 1.
    θ = get_θ(p0)
    
    # Cycle gibbs sampling over assignments of each Ca²⁺ state, ie each component of j indexed by ℓ
    for ℓ=1:8
        # 2.
        tmp1 = @SVector [sum(ws[i].*pdf.(Ns,xs[ℓ])) for i=1:8]
        
        # 3.
        if ℓ<= 4
                tmp2 = @SVector [sum(ws[i].*cdf.(Ns,ys[ℓ])) for i=1:8]
                tmp3 = @SVector [sum(ws[i].*pdf.(Ns,ys[ℓ])) for i=1:8]
                    
            tmp1 = tmp1.*(1 .- tmp2).^7 .*tmp3
        end
        
        # normalize the distribution and sample
        λ = θ.*tmp1; λ = mypos.(λ); Λ = sum(λ); λ = λ/Λ
        C = Categorical(λ)
        
        jℓ = rand(rng,C)
        set_j!(p0,jℓ;k=ℓ)
    end
end;

## Run MCMC

In [ ]:
# MCMC runs
nchains = 4; 
nsmps = 100000;

# Burn in and chain splitting parameters
burn = nsmps÷2
split = nsmps÷4

# Plotting parameters
nmcmcstep = nsmps÷25000;
ntracestep = nsmps÷1000;

In [ ]:
SMPS = Vector{Matrix{Float64}}(undef,nchains)
p0 = Vector{Float64}(undef,98); smps = Matrix{Float64}(undef,98,nsmps);
aptrates = Vector{Dict{Symbol,Float64}}(undef,nchains)
for chn=1:nchains
    println("Beginning chain $(chn)...")
    gibbssmp_p0!(rng,p0)
    smps[:,1] .= p0
    aptrates[chn] = Dict{Symbol,Float64}(:μ=>0,:log10σsq=>0,:α=>0,:w=>0,:β=>0)
    
    prg = 0.; δprg = 0.05
    for ℓ=2:nsmps
        # cycle kernels
        aptrates[chn][:μ] += gibbssmp_μ!(rng,p0)
        aptrates[chn][:log10σsq] += gibbssmp_log10σsq!(rng,p0)
        aptrates[chn][:α] += gibbssmp_α!(rng,p0)
        aptrates[chn][:w] += gibbssmp_w!(rng,p0)
        aptrates[chn][:β] += gibbssmp_β!(rng,p0)
        gibbssmp_θ!(rng,p0)
        gibbssmp_j!(rng,p0)
        
        smps[:,ℓ] .= p0
        
        while ℓ/nsmps >= prg + δprg
            prg += δprg
            println("Progress through samples $(prg)")
        end
    end
    SMPS[chn] = copy(smps)
    println("")
end

for chn=1:nchains
    for key in keys(aptrates[chn])
        aptrates[chn][key] *= 1/nsmps
    end
end;

### Analyze results
Parameters are ordered like $p = \left(\vec{\mu},\log_{10}\vec{\sigma}^2,\alpha,\vec{w}_1,\ldots,\vec{w}_8,\beta,\vec{\theta},\vec{j}\right)$.

#### MH acceptance rates

In [ ]:
prms = [key for key in keys(aptrates[1])];
M = [aptrates[j][prms[i]] for i=1:length(prms),j=1:nchains]
dfapt = DataFrame(hcat(prms,M),["parameters",["chain $(i)" for i=1:nchains]...])

#### Gelman-Rubin convergence statistic

In [ ]:
# Discard burn-in
for chn=1:nchains
    SMPS[chn] = SMPS[chn][:,burn+1:end]
end

In [ ]:
# Split into half-chains
HALF = vcat(SMPS,SMPS)
for ℓ=1:2nchains
    if ℓ<=nchains
        HALF[ℓ] = HALF[ℓ][:,1:split]
    else
        HALF[ℓ] = HALF[ℓ][:,(split+1):(2split)]
    end
end;

In [ ]:
function gr̂(HALF...)
    nchains = length(HALF); nprm,nsmps = size(HALF[1])
    # sample averages within a chain
    xbar = Matrix{Float64}(undef,nprm,nchains)
    for ℓ=1:nchains
        xbar[:,ℓ] = sum(HALF[ℓ],dims=2)/nsmps
    end
    
    # overall sample average across all chains
    xbarbar = sum(xbar,dims=2)/nchains
    
    # sample variances within a chain
    var = Matrix{Float64}(undef,nprm,nchains)
    for ℓ=1:nchains
        var[:,ℓ] = 1/(nsmps-1)*sum((HALF[ℓ] .-xbar[:,ℓ]).^2,dims=2)
    end
    
    # average sample variance within a chain
    W = sum(var,dims=2)/nchains
    
    # Between chain variance (ie variance of the chain sample means)
    varbar = 1/(nchains-1)*sum((xbar .- xbarbar).^2,dims=2)
    B = nsmps*varbar
    
    R̂ = (nsmps-1)/nsmps*W+1/nsmps*B
    R̂ = .√(R̂./W)
    
    return R̂
end;

In [ ]:
R̂ = gr̂(HALF...);

In [ ]:
# Other parameters may have poor Rhat's because of the mixture labeling problem
dfrhat = DataFrame(hcat([:α,:β],R̂[[17,82]]),["parameters","Rhat"])

#### Posterior histograms and trace plots

In [ ]:
# stack half chains into one
smps = hcat(HALF...);

In [ ]:
αs = [get_α(smps[:,ℓ]) for ℓ=1:nsmps]
p1_α = histogram(αs[1:nmcmcstep:end],labels="",normalize=:pdf,alpha=0.5,linewidth=0,linecolor=:white,
                 xlabel="α",ylabel="pdf")
p2_α = plot(αs[1:ntracestep:end],labels="",ylabel="α",xlabel="sample")
plot(p1_α,p2_α,size=(700,200),margin=4mm)

In [ ]:
βs = [get_β(smps[:,ℓ]) for ℓ=1:nsmps]
p1_β = histogram(βs[1:nmcmcstep:end],labels="",normalize=:pdf,alpha=0.5,linewidth=0,linecolor=:white,
                 xlabel="β",ylabel="pdf")
p2_β = plot(βs[1:ntracestep:end],labels="",ylabel="β",xlabel="sample")
plot(p1_β,p2_β,size=(700,200),margin=4mm)

In [ ]:
js = [length(unique([get_j(smps[:,ℓ];k=i) for i=1:8])) for ℓ=1:nsmps]
p1_js = histogram(js[1:nmcmcstep:end],labels="",normalize=:probability,alpha=0.5,linewidth=0,linecolor=:white,
                  xlabel="# js",ylabel="pmf")
    p2_js = plot(js[1:ntracestep:end],labels="",ylabel="# js",xlabel="sample")
plot(p1_js,p2_js,size=(700,200),margin=4mm)

In [ ]:
ufrc = Matrix{Float64}(undef,8,nsmps)
for ℓ=1:nsmps
    p0 = @view smps[:,ℓ]
    jℓ = @SVector [get_j(p0;k=i) for i=1:8]
    
    # sample unfolding force for each Ca2+ state
    for i=1:8
        # grab weights associated to this state
        ws = get_w(p0;k=Int64(jℓ[i]))
        
        # decide which component to sample
        C = Categorical(ws)
        id = rand(rng,C)
        
        # grab hyperparameters for this normal component
        μid = get_μ(p0;k=id); log10σsqid = get_log10σsq(p0;k=id); σid = √(10^(log10σsqid))
        ufrc[i,ℓ] = μid + σid*randn(rng)
    end
end;     

#### $\mathrm{Ca}^{2+}$ state posterior forces for EC19-20

In [ ]:
lbl = ["p000","p001","p011","p111","p010","p100","p101","p110"]
p1_ufrc = [histogram(ufrc[i,1:nmcmcstep:end],normalize=:pdf,alpha=0.5,linecolor=:white,linewidth=0,labels=lbl[i],
           xlabel="force (pN)",ylabel="density") for i=1:8]
for ℓ=1:8
    scatter!(p1_ufrc[ℓ],[xs[ℓ]],[0],markersize=6,marker=:o,labels="")
end
p2_ufrc = [plot(ufrc[i,1:ntracestep:end],labels=lbl[i],
           ylabel="force (pN)",xlabel="sample") for i=1:8];

In [ ]:
lay = @layout [a b c d;e f g h]
plot(p1_ufrc...,layout=lay,margin=10mm,size=(4*450,500))

In [ ]:
savefig("unfoldingforce_byevent.png");
savefig("unfoldingforce_byevent.pdf");

In [ ]:
lay = @layout [a b c d;e f g h]
plot(p2_ufrc...,layout=lay,margin=10mm,size=(4*450,500))

#### Unfolding force histograms for paper

In [ ]:
p3_ufrc =  plot()
for ℓ=1:4
    histogram!(ufrc[ℓ,1:nmcmcstep:end],normalize=:pdf,alpha=0.5,linecolor=:white,linewidth=0,labels=lbl[ℓ],
              xlabel="force (pN)",ylabel="density",size=(350,200),xlims=(500,1500),ylims=(0,0.0075),legend=:topleft)
end
plot!(p3_ufrc,inset=(1,bbox(0,0,0.475,0.325,:top,:right)));
for ℓ=1:4
    histogram!(p3_ufrc[2],ufrc[ℓ,1:nmcmcstep:end],normalize=:pdf,alpha=0.5,linecolor=:white,linewidth=0,labels="",xlims=(500,1500))
end


p4_ufrc = plot()
for ℓ=5:8
    histogram!(ufrc[ℓ,1:nmcmcstep:end],normalize=:pdf,alpha=0.5,linecolor=:white,linewidth=0,labels=lbl[ℓ],
              xlabel="force (pN)",ylabel="density",size=(350,200),xlims=(500,1500),ylims=(0.000,0.0075),legend=:bottomright)
end
plot!(p4_ufrc,inset=(1,bbox(0,0,0.475,0.325,:top,:right)));
for ℓ=5:8
    histogram!(p4_ufrc[2],ufrc[ℓ,1:nmcmcstep:end],normalize=:pdf,alpha=0.5,linecolor=:white,linewidth=0,labels="",xlims=(500,1500))
end

In [ ]:
plot(p3_ufrc,size=(450,300),margin=4mm)

In [ ]:
savefig("unfoldingforceposterior_typicals.png");
savefig("unfoldingforceposterior_typicals.pdf");

In [ ]:
plot(p4_ufrc,size=(450,300),margin=4mm)

In [ ]:
savefig("unfoldingforceposterior_atypicals.png");
savefig("unfoldingforceposterior_atypicals.pdf");

#### $\mathrm{Ca}^{2+}$ state posterior forces for EC17-25

In [ ]:
NL = 8
ufrc_1725 = Matrix{Float64}(undef,4,nsmps)
for ℓ=1:nsmps
    p0 = @view smps[:,ℓ]
    jℓ = @SVector [get_j(p0;k=i) for i=1:8]
    
    # sample unfolding force for each Ca2+ state
    for i=1:4
        # grab weights associated to this state
        ws = get_w(p0;k=Int64(jℓ[i]))
        C = Categorical(ws)
        
        mnr = Inf
        for j=1:NL
            # decide which component to sample
            id = rand(rng,C)
            
            # grab hyperparameters for this normal component
            μid = get_μ(p0;k=id); log10σsqid = get_log10σsq(p0;k=id); σid = √(10^(log10σsqid))
            
            # see if this component is new min
            trial = μid + σid*randn(rng)
            mnr = trial < mnr ? trial : mnr
        end
        
        ufrc_1725[i,ℓ] = mnr 
    end
end;

In [ ]:
lbl = ["p000","p001","p011","p111"]
p1_ufrc_1725 = [histogram(ufrc_1725[i,1:nmcmcstep:end],normalize=:pdf,alpha=0.5,linecolor=:white,linewidth=0,labels=lbl[i],
           xlabel="force (pN)",ylabel="density") for i=1:4]
for ℓ=1:4
    scatter!(p1_ufrc_1725[ℓ],[ys[ℓ]],[0],markersize=6,marker=:o,labels="")
end
p2_ufrc_1725 = [plot(ufrc_1725[i,1:ntracestep:end],labels=lbl[i],
           ylabel="force (pN)",xlabel="density") for i=1:4];

In [ ]:
lay = @layout [a b c d]
plot(p1_ufrc_1725...,layout=lay,margin=10mm,size=(4*450,250))

In [ ]:
savefig("unfoldingforceEC17-25_byevent.png");
savefig("unfoldingforceEC17-25_byevent.pdf");

#### $\hat{R}$ for posterior predictive unfolding force distributions at each $\mathrm{Ca}^{2+}$ configuration

In [ ]:
FSMPS = similar(HALF)
ufrc = Matrix{Float64}(undef,8,split)
for c=1:2nchains
    tmp = HALF[c]
    ntmp = size(tmp)[2]
    for ℓ=1:ntmp
        p0 = @view tmp[:,ℓ]
        jℓ = @SVector [get_j(p0;k=i) for i=1:8]
        
        # sample unfolding force for each Ca2+ state
        for i=1:8
            # grab weights associated to this state
            ws = get_w(p0;k=Int64(jℓ[i]))
                
            # decide which component to sample
            C = Categorical(ws)
            id = rand(rng,C)
            
            # grab hyperparameters for this normal component
            μid = get_μ(p0;k=id); log10σsqid = get_log10σsq(p0;k=id); σid = √(10^(log10σsqid))
            ufrc[i,ℓ] = μid + σid*randn(rng)
        end
    end
    FSMPS[c] = copy(ufrc)
end; 

In [ ]:
lbl = ["p000","p001","p011","p111","p010","p100","p101","p110"]
R̂ = gr̂(FSMPS...)
dfrhat = DataFrame(hcat(lbl,R̂),["Ca²⁺ configurations","R̂"])

#### Export data for force unfoldings

In [ ]:
smps_μ = Matrix{Float64}(undef,8,nsmps)
smps_σ = Matrix{Float64}(undef,8,nsmps)
smps_j = Matrix{Float64}(undef,8,nsmps)
smps_w = Matrix{Float64}(undef,64,nsmps)
for ℓ=1:nsmps
    p0 = @view smps[:,ℓ]
    smps_μ[:,ℓ] .= [get_μ(p0;k=i) for i=1:8]
    smps_σ[:,ℓ] .= .√(10 .^([get_log10σsq(p0;k=i) for i=1:8]))
    smps_j[:,ℓ] .= [get_j(p0;k=i) for i=1:8]
    for blk=1:8
        smps_w[8*(blk-1)+1:8*blk,ℓ] .= get_w(p0;k=blk)
    end
end

In [ ]:
# reorder to match Ca2+ site probabilities notebook:
#  current order: p000 p001 p011 p111 p010 p100 p101 p110
#  new order: p010 p100 p101 p110 p000 p001 p011 p111
# you only need to change the j ordering because the mixture events are
# independent of how Ca2+ configurations are assigned to them.
σperm = [5,6,7,8,1,2,3,4]
for ℓ=1:nsmps
    smps_j[:,ℓ] = smps_j[σperm,ℓ]
end
M = vcat(smps_μ,smps_σ,smps_j,smps_w);

In [ ]:
df = DataFrame(M,:auto);
CSV.write("gibbssmps_unfoldinghypers.csv",df);

### Restore the relevant sample values for regenerating figure plots from earlier MCMC runs

In [ ]:
df = CSV.read("gibbssmps_unfoldinghypers.csv",DataFrame); rng = MersenneTwister();
nsmps = ncol(df)
smps = Matrix{Float64}(undef,98,nsmps);
M_μσjw = [df[i,j] for i=1:nrow(df),j=1:ncol(df)];

In [ ]:
for ℓ=1:nsmps
    # initialize the irrelevant variables
    p0 = @view smps[:,ℓ]
    gibbssmp_p0!(rng,p0)

    # restore the relevant hypers
    for blk=1:8
        set_μ!(p0,M_μσjw[blk,ℓ];k=blk)
        shift = 8
        set_log10σsq!(p0,2*log10(M_μσjw[shift+blk,ℓ]);k=blk)
        shift = 24
        set_w!(p0,M_μσjw[shift+(blk-1)*8+1:shift+blk*8,ℓ];k=blk)
    end

    # handle the cluster assignment permutation between notebooks (ie revert back to order of this notebook)
    shift = 16
    σperm = [5,6,7,8,1,2,3,4]
    js = M_μσjw[shift+1:shift+8,ℓ]
    js = js[σperm]
    for blk=1:8
        set_j!(p0,js[blk];k=blk)
    end 
end 

In [ ]:
ufrc = Matrix{Float64}(undef,8,nsmps)
for ℓ=1:nsmps
    p0 = @view smps[:,ℓ]
    jℓ = @SVector [get_j(p0;k=i) for i=1:8]
    
    # sample unfolding force for each Ca2+ state
    for i=1:8
        # grab weights associated to this state
        ws = get_w(p0;k=Int64(jℓ[i]))
        
        # decide which component to sample
        C = Categorical(ws)
        id = rand(rng,C)
        
        # grab hyperparameters for this normal component
        μid = get_μ(p0;k=id); log10σsqid = get_log10σsq(p0;k=id); σid = √(10^(log10σsqid))
        ufrc[i,ℓ] = μid + σid*randn(rng)
    end
end;     

In [ ]:
lbl = ["p000","p001","p011","p111","p010","p100","p101","p110"]
p1_ufrc = [histogram(ufrc[i,1:nmcmcstep:end],normalize=:pdf,alpha=0.5,linecolor=:white,linewidth=0,labels=lbl[i],
           xlabel="force (pN)",ylabel="density") for i=1:8]
for ℓ=1:8
    scatter!(p1_ufrc[ℓ],[xs[ℓ]],[0],markersize=6,marker=:o,labels="")
end
p2_ufrc = [plot(ufrc[i,1:ntracestep:end],labels=lbl[i],
           ylabel="force (pN)",xlabel="sample") for i=1:8];

In [ ]:
lay = @layout [a b; c d;e f; g h]
plot(p1_ufrc...,layout=lay,margin=10mm,size=(2*450,2*500))

In [ ]:
savefig("unfoldingforce_byevent_reload.png");
savefig("unfoldingforce_byevent_reload.pdf");

In [ ]:
lay = @layout [a b c d;e f g h]
plot(p2_ufrc...,layout=lay,margin=10mm,size=(4*450,500))

In [ ]:
p3_ufrc =  plot()
for ℓ=1:4
    histogram!(ufrc[ℓ,1:nmcmcstep:end],normalize=:pdf,alpha=0.5,linecolor=:white,linewidth=0,labels=lbl[ℓ],
              xlabel="force (pN)",ylabel="density",size=(350,200),xlims=(500,1500),ylims=(0,0.0075),legend=:topleft)
end
plot!(p3_ufrc,inset=(1,bbox(0,0,0.475,0.325,:top,:right)));
for ℓ=1:4
    histogram!(p3_ufrc[2],ufrc[ℓ,1:nmcmcstep:end],normalize=:pdf,alpha=0.5,linecolor=:white,linewidth=0,labels="",xlims=(500,1500))
end


p4_ufrc = plot()
for ℓ=5:8
    histogram!(ufrc[ℓ,1:nmcmcstep:end],normalize=:pdf,alpha=0.5,linecolor=:white,linewidth=0,labels=lbl[ℓ],
              xlabel="force (pN)",ylabel="density",size=(350,200),xlims=(500,1500),ylims=(0.000,0.0075),legend=:bottomright)
end
plot!(p4_ufrc,inset=(1,bbox(0,0,0.475,0.325,:top,:right)));
for ℓ=5:8
    histogram!(p4_ufrc[2],ufrc[ℓ,1:nmcmcstep:end],normalize=:pdf,alpha=0.5,linecolor=:white,linewidth=0,labels="",xlims=(500,1500))
end

In [ ]:
plot(p3_ufrc,size=(450,300),margin=4mm)

In [ ]:
savefig("unfoldingforceposterior_typicals_reload.png");
savefig("unfoldingforceposterior_typicals_reload.pdf");

In [ ]:
plot(p4_ufrc,size=(450,300),margin=4mm)

In [ ]:
savefig("unfoldingforceposterior_atypicals_reload.png");
savefig("unfoldingforceposterior_atypicals_reload.pdf");

In [ ]:
NL = 8
ufrc_1725 = Matrix{Float64}(undef,4,nsmps)
for ℓ=1:nsmps
    p0 = @view smps[:,ℓ]
    jℓ = @SVector [get_j(p0;k=i) for i=1:8]
    
    # sample unfolding force for each Ca2+ state
    for i=1:4
        # grab weights associated to this state
        ws = get_w(p0;k=Int64(jℓ[i]))
        C = Categorical(ws)
        
        mnr = Inf
        for j=1:NL
            # decide which component to sample
            id = rand(rng,C)
            
            # grab hyperparameters for this normal component
            μid = get_μ(p0;k=id); log10σsqid = get_log10σsq(p0;k=id); σid = √(10^(log10σsqid))
            
            # see if this component is new min
            trial = μid + σid*randn(rng)
            mnr = trial < mnr ? trial : mnr
        end
        
        ufrc_1725[i,ℓ] = mnr 
    end
end;

In [ ]:
lbl = ["p000","p001","p011","p111"]
p1_ufrc_1725 = [histogram(ufrc_1725[i,1:nmcmcstep:end],normalize=:pdf,alpha=0.5,linecolor=:white,linewidth=0,labels=lbl[i],
           xlabel="force (pN)",ylabel="density") for i=1:4]
for ℓ=1:4
    scatter!(p1_ufrc_1725[ℓ],[ys[ℓ]],[0],markersize=6,marker=:o,labels="")
end
p2_ufrc_1725 = [plot(ufrc_1725[i,1:ntracestep:end],labels=lbl[i],
           ylabel="force (pN)",xlabel="density") for i=1:4];

In [ ]:
lay = @layout [a b; c d]
plot(p1_ufrc_1725...,layout=lay,margin=10mm,size=(2*450,2*250))

In [ ]:
savefig("unfoldingforceEC17-25_byevent_reload.png");
savefig("unfoldingforceEC17-25_byevent_reload.pdf");